In [2]:
import os
import csv

def create_labels(image_folder, label):
    with open('labels.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for filename in os.listdir(image_folder):
            if filename.endswith(('.jpg', '.png')):
                writer.writerow([filename, label])

# Paths to your image folders
solar_farm_folder = './Users/macbookpro/Desktop/SolarProject/SolarFarm_Images'
non_solar_farm_folder = './Users/macbookpro/Desktop/SolarProject/Non_SolarFarms'

# Create labels
create_labels(solar_farm_folder, 1)  # Label 1 for solar farms
create_labels(non_solar_farm_folder, 0)  # Label 0 for non-solar farms


FileNotFoundError: [Errno 2] No such file or directory: './Users/macbookpro/Desktop/SolarProject/SolarFarm_Images'